In [1]:
import os
import tensorflow as tf
import numpy as np
import glob
import pandas as pd

/home/nikhila/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nikhila/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/nikhila/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nikhila/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nikhila/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed

In [2]:
import matplotlib.pyplot as plt

In [3]:
#reading files

filenames = glob.glob("/home/nikhila/Quantiphi/all/train/*.jpg")
filenames.sort()

In [4]:
len(filenames)

10222

In [5]:
filenames[0:2]

['/home/nikhila/Quantiphi/all/train/000bec180eb18c7604dcecc8fe0dba07.jpg',
 '/home/nikhila/Quantiphi/all/train/001513dfcb2ffafc82cccf4d8bbaba97.jpg']

In [6]:
labels_file = pd.read_csv('/home/nikhila/Quantiphi/all/labels.csv',sep=',')

In [7]:
labels_file.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [8]:
labels_file.dtypes

id       object
breed    object
dtype: object

In [9]:
unq_labels = np.unique(labels_file['breed'].values)

In [10]:
#unique labels to unique values to make one hot encoding with tfdata

#SOURCE

#http://benalexkeen.com/mapping-categorical-data-in-pandas/

uniq_vals = labels_file.breed.astype("category", categories = unq_labels).cat.codes

/home/nikhila/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [11]:
labels = uniq_vals.values

In [12]:
labels

array([19, 37, 85, ...,  3, 75, 28], dtype=int8)

In [26]:
list_labels = list(labels)

In [28]:
len(list_labels)

10222

In [25]:
type(labels)

numpy.ndarray

In [24]:
labels.shape

(10222,)

In [37]:
list_labels = tf.one_hot(list_labels,120)

In [38]:
dataset = tf.data.Dataset.from_tensor_slices((filenames,list_labels))

In [39]:
def parse_function(filename,label):
    image_string = tf.read_file(filename)
    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string,channels=3)
    #this will convert image values to float values in [0,1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    resized_image = tf.image.resize_images(image,[224,224])
    
    return resized_image,label

In [40]:
#def train_preprocess(image, label):
 #   image = tf.image.random_brightness
  #  image = tf.image.random_saturation(image, lower=0.5, upper=1.5)

    # Make sure the image is still in [0, 1]
   # image = tf.clip_by_value(image, 0.0, 1.0)

    #return image, label

In [41]:
batch_size = 32

In [42]:
dataset = dataset.map(parse_function)

In [43]:
dataset = dataset.batch(batch_size)

In [44]:
dataset = dataset.prefetch(1)

In [45]:
iterator = dataset.make_initializable_iterator()
x,y = iterator.get_next()

In [46]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [47]:
#model
learning_rate = 0.0001
num_batches = int(len(filenames)/batch_size)

conv1 = tf.layers.conv2d(x, 32, 3, strides = 1,activation=tf.nn.relu)
conv1 = tf.layers.max_pooling2d(conv1, 2, 2,padding='same')
conv1 = tf.layers.batch_normalization(conv1)


conv2 = tf.layers.conv2d(conv1, 64, 5,strides=1, activation=tf.nn.relu)
conv2 = tf.layers.max_pooling2d(conv2, 2, 2,padding='same')
conv2 = tf.layers.batch_normalization(conv2)


conv3 = tf.layers.conv2d(conv2,128, 5,strides=1, activation=tf.nn.relu)
conv3 = tf.layers.max_pooling2d(conv3, 2, 2)
conv3 = tf.layers.batch_normalization(conv3)

#flatten layer
fc1 = tf.contrib.layers.flatten(conv3)

#fully connected dense layer
fc1 = tf.layers.dense(fc1,2048)

fc2 = tf.layers.dense(fc1,512)

logits_train = tf.layers.dense(fc2,120)

#pred = tf.nn.softmax(logits_train)

predict_classes = tf.argmax(logits_train, axis = 1)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_train,labels=y))

train_op = optimizer.minimize(loss_op)

compare = tf.equal(predict_classes,tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(compare,tf.float32))



In [48]:
with tf.Session() as sess:
    n_epochs = 100
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        sess.run(iterator.initializer)
        for j in range(num_batches):
            _, loss_val,acc = sess.run([train_op, loss_op, accuracy])
            if j%10 == 0:
                print ("batch: ",j," loss: ",loss_val, "accuracy:", acc)
        print ("epoch: ",i," loss: ",loss_val, "accuracy:", acc)    

batch:  0  loss:  4.7569504 accuracy: 0.0625
batch:  10  loss:  4.9778156 accuracy: 0.03125
batch:  20  loss:  4.7804747 accuracy: 0.03125
batch:  30  loss:  4.8544836 accuracy: 0.0
batch:  40  loss:  4.7974496 accuracy: 0.03125
batch:  50  loss:  4.760803 accuracy: 0.0
batch:  60  loss:  4.7447214 accuracy: 0.03125
batch:  70  loss:  4.7597055 accuracy: 0.0
batch:  80  loss:  4.779726 accuracy: 0.0
batch:  90  loss:  4.736282 accuracy: 0.0
batch:  100  loss:  4.7319546 accuracy: 0.0
batch:  110  loss:  4.8164177 accuracy: 0.03125
batch:  120  loss:  4.724188 accuracy: 0.0
batch:  130  loss:  4.695612 accuracy: 0.0625
batch:  140  loss:  4.470126 accuracy: 0.0625
batch:  150  loss:  4.807123 accuracy: 0.0
batch:  160  loss:  4.528813 accuracy: 0.03125
batch:  170  loss:  4.4893622 accuracy: 0.0625
batch:  180  loss:  4.5710745 accuracy: 0.0
batch:  190  loss:  4.8603015 accuracy: 0.0
batch:  200  loss:  4.59149 accuracy: 0.0
batch:  210  loss:  4.665 accuracy: 0.03125
batch:  220  loss

batch:  160  loss:  0.26558954 accuracy: 0.9375
batch:  170  loss:  0.1573348 accuracy: 1.0
batch:  180  loss:  0.123371296 accuracy: 1.0
batch:  190  loss:  0.0969233 accuracy: 0.96875
batch:  200  loss:  0.06420777 accuracy: 1.0
batch:  210  loss:  0.19702192 accuracy: 0.9375
batch:  220  loss:  0.17542401 accuracy: 0.9375
batch:  230  loss:  0.0741895 accuracy: 1.0
batch:  240  loss:  0.11282713 accuracy: 1.0
batch:  250  loss:  0.21360675 accuracy: 0.90625
batch:  260  loss:  0.11646328 accuracy: 0.9375
batch:  270  loss:  0.1723974 accuracy: 0.9375
batch:  280  loss:  0.09717065 accuracy: 0.96875
batch:  290  loss:  0.14097321 accuracy: 0.96875
batch:  300  loss:  0.015138797 accuracy: 1.0
batch:  310  loss:  0.164006 accuracy: 0.96875
batch:  0  loss:  0.026214745 accuracy: 1.0
batch:  10  loss:  0.07211636 accuracy: 1.0
batch:  20  loss:  0.051663823 accuracy: 1.0
batch:  30  loss:  0.17636123 accuracy: 0.9375
batch:  40  loss:  0.06561159 accuracy: 1.0
batch:  50  loss:  0.0415

KeyboardInterrupt: 